# Progetto di Social Computing

a.a. 2022/2023

## Attività preliminari

### Librerie e costanti

In [2]:
# Import delle librerie utilizzate
import os, tweepy, json
import networkx as nx 
import pandas as pd 
import numpy as np 
import random as rn
import networkx as nx
import matplotlib.pyplot as plt 

In [3]:
# Cartelle di salvataggio
data_folder = "./data"
out_folder = "./out"

### Funzioni

In [4]:
# Salvataggio in locale
def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    
    with open(f"{folder}/{filename}", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent = 4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

In [5]:
# Lettura da locale
def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

In [16]:
# Somma dei numeri di tweet prodotti in un intervallo di tempo
def sum_tweets_count(tweet_groups):
    sum = 0

    for tweet_count in tweet_groups:
        sum += tweet_count["tweet_count"]
    
    return sum

### Credenziali Twitter API

In [6]:
# Caricamento credenziali da JSON
api_access = read_json("./api_access.json")

Data read from path: ./api_access.json


## Recupero dei follower e dei follower dei follower

### Recupero dei follower

Si vogliono recuperare, utilizzando la libreria `tweepy`, tutti i follower dell'utente *@KevinRoitero*, corredati delle seguenti informazioni:

* attributi di default;
* descrizione del profilo;
* metriche pubbliche dell'account;
* se l'account è protetto

In [18]:
 # Si inizializza il client
client = tweepy.Client(bearer_token=api_access["bearer_token"])

username = "KevinRoitero"
all_user_followers = []

# Si recuperano e memorizzano le informazioni dell'utente
response = client.get_user(username = username, user_fields=["description", "protected", "public_metrics"])
user = dict(response.data)

# Si recuperano e memorizzano i follower dell'utente
response = client.get_users_followers(user["id"], user_fields=["description", "protected", "public_metrics"], 
                                      max_results=150) # max_results = 150 perché i follower dell'utente sono nell'ordine di 130
for follower in response.data:
    all_user_followers.append(dict(follower))

# Si associano i follower trovati all'utente di partenza
user["followers"] = all_user_followers

# Si serializza su JSON il risultato ottenuto
serialize_json(data_folder, "user_followers.json", user)

Data serialized to path: ./data/user_followers.json


### Aggiunta del numero di tweet prodotti nell'ultima settimana

Ai follower trovati, si vuole aggiungere il numero di tweet pubblicati nell'ultima settimana. Per avere uniformità, troveremo anche il numero di tweet pubblicati nell'ultima settimana dall'utente *@KevinRoitero*.

In [19]:
# Si sceglie l'utilizzo dello stesso client di partenza, o comunque senza la possibilità di mettersi in attesa, 
# perché numero_richieste < 300, dove 300 è il numero massimo di richieste per l'endpoint get_recent_tweets_count()
client = tweepy.Client(bearer_token=api_access["bearer_token"])

# Si caricano i dati dell'utente e quelli dei suoi follower
user = read_json(data_folder+"/user_followers.json")

# Si memorizza il numero di tweet pubblicati dall'utente nell'ultima settimana
response = client.get_recent_tweets_count(query="from:"+user["username"], granularity="day")
user["last_week_tweets_count"] = sum_tweets_count(response.data)

# Si ripete lo stesso per i follower dell'utente
for follower in user["followers"]:
    if(not follower["protected"]): # non si può accedere ai tweet di un utente 'protected'
        response = client.get_recent_tweets_count(query="from:"+follower["username"], granularity="day")
        follower["last_week_tweets_count"] = sum_tweets_count(response.data)

# Si serializza su JSON il risultato ottenuto
serialize_json(data_folder, "followers_last_week_tweets.json", user)

Data read from path: ./data/user_followers.json
Data serialized to path: ./data/followers_last_week_tweets.json


### Recupero dei follower dei follower

Per ciascun follower di *@KevinRoitero* avente almeno 1 follower e non `protected`, si vogliono scaricare le seguenti informazioni:

* attributi di default;
* descrizione del profilo;
* metriche pubbliche dell'account;
* se l'account è protetto

L'approccio che è stato seguito è quello di un download *parallelo*, ossia ciascun collaboratore al progetto si è preso in carico di scaricare una porzione di follower dei follower di *@KevinRoitero*.

In [7]:
# Si impone al client di attendere nel caso di raggiungimento del limite delle richieste
client = tweepy.Client(bearer_token=api_access["bearer_token"], wait_on_rate_limit=True)

# Si caricano i dati dell'utente e quelli dei suoi follower
user = read_json(data_folder+"/followers_last_week_tweets.json")

# Parallelizzando il calcolo, si decide di partizionare tra collaboratori il numero di follower
followers = np.array(user["followers"])

f_num = user["public_metrics"]["followers_count"]
start = f_num*api_access["id"] // 4
end = f_num*(api_access["id"]+1) // 4 

# Si scaricano i follower dei follower nell'intervallo del collaboratore
for i in range(start, end):
    follower = followers[i]
    
    follower["followers"] = []

    if (not follower["protected"] and follower["public_metrics"]["followers_count"] > 0):
        for all_follower_followers in tweepy.Paginator(client.get_users_followers, id = follower["id"], user_fields=["description", "protected", "public_metrics"], max_results = 1000):
            parsed_followers = []
            
            for ff in all_follower_followers.data:
                # L'oggetto User deve essere interpretato
                parsed_follower = {
                    "id" : ff["id"],
                    "public_metrics" : ff["public_metrics"],
                    "description" : ff["description"],
                    "name" : ff["name"],
                    "protected" : ff["protected"],
                    "username" : ff["username"]
                }
                parsed_followers.append(parsed_follower)
            
            follower["followers"] += parsed_followers

# Si salva la porzione di follower scaricata
serialize_json(data_folder, "f_of_f_"+str(api_access["id"]), list(followers))

Data read from path: ./data/followers_last_week_tweets.json


Rate limit exceeded. Sleeping for 891 seconds.
Rate limit exceeded. Sleeping for 890 seconds.
Rate limit exceeded. Sleeping for 888 seconds.
Rate limit exceeded. Sleeping for 887 seconds.
Rate limit exceeded. Sleeping for 888 seconds.
Rate limit exceeded. Sleeping for 886 seconds.
Rate limit exceeded. Sleeping for 887 seconds.
Rate limit exceeded. Sleeping for 887 seconds.
Rate limit exceeded. Sleeping for 888 seconds.
Rate limit exceeded. Sleeping for 887 seconds.
Rate limit exceeded. Sleeping for 888 seconds.
Rate limit exceeded. Sleeping for 886 seconds.
Rate limit exceeded. Sleeping for 886 seconds.
Rate limit exceeded. Sleeping for 887 seconds.
Rate limit exceeded. Sleeping for 887 seconds.
Rate limit exceeded. Sleeping for 887 seconds.
Rate limit exceeded. Sleeping for 888 seconds.
Rate limit exceeded. Sleeping for 888 seconds.
Rate limit exceeded. Sleeping for 888 seconds.
Rate limit exceeded. Sleeping for 887 seconds.
Rate limit exceeded. Sleeping for 886 seconds.
Rate limit ex

TypeError: Object of type ndarray is not JSON serializable